In [10]:
import numpy as np
import re

In [42]:
import numpy as np
import re

def float_to_fixed_point(value, bit_width, fractional_bits):
    """
    Convert a floating-point value to fixed-point representation.
    
    Args:
        value (float): The floating-point value to convert.
        bit_width (int): Total bit width of the fixed-point number.
        fractional_bits (int): Number of bits for the fractional part.
    
    Returns:
        int: The fixed-point representation as an integer.
    """
    # Calculate the scaling factor
    scale = 2 ** fractional_bits
    
    # Clip the value to the representable range
    max_val = (2 ** (bit_width - 1) - 1) / scale
    min_val = - (2 ** (bit_width - 1)) / scale
    value = np.clip(value, min_val, max_val)
    
    # Convert to fixed-point
    fixed_point = int(round(value * scale))
    
    return fixed_point

def parse_conv_weights_and_biases(file_path):
    """
    Parse the convolutional weights and biases from the .h file.
    
    Args:
        file_path (str): Path to the .h file.
    
    Returns:
        tuple: (conv_weights, conv_biases) as numpy arrays.
    """
    with open(file_path, 'r') as f:
        content = f.read()
    
    # Extract convolutional weights using regex
    conv_weights_pattern = re.compile(r"float conv_weights\s*\[FILTERS\]\[KRN_ROWS\]\[KRN_COLS\]\s*=\s*\{(.*?)\};", re.DOTALL)
    conv_weights_match = conv_weights_pattern.search(content)
    if not conv_weights_match:
        raise ValueError("Convolutional weights not found in the file.")
    
    conv_weights_str = conv_weights_match.group(1)
    conv_weights = []
    for filter_str in conv_weights_str.split("},"):
        filter_weights = []
        for row_str in filter_str.split("},"):
            row_weights = []
            for val_str in re.findall(r"-?\d+\.\d+", row_str):
                row_weights.append(float(val_str))
            if row_weights:
                filter_weights.append(row_weights)
        if filter_weights:
            conv_weights.append(filter_weights)
    
    conv_weights = np.array(conv_weights)
    
    # Extract convolutional biases using regex
    conv_biases_pattern = re.compile(r"float conv_biases\s*\[FILTERS\]\s*=\s*\{(.*?)\};", re.DOTALL)
    conv_biases_match = conv_biases_pattern.search(content)
    if not conv_biases_match:
        raise ValueError("Convolutional biases not found in the file.")
    
    conv_biases_str = conv_biases_match.group(1)
    conv_biases = np.array([float(val_str) for val_str in re.findall(r"-?\d+\.\d+", conv_biases_str)])
    
    return conv_weights, conv_biases

def parse_dense_weights_and_biases(file_path):
    """
    Parse the dense weights and biases from the .h file.
    
    Args:
        file_path (str): Path to the .h file.
    
    Returns:
        tuple: (dense_weights, dense_biases) as numpy arrays.
    """
    with open(file_path, 'r') as f:
        content = f.read()
    
    # Extract dense weights using regex
    dense_weights_pattern = re.compile(r"float dense_weights\s*\[FLAT_SIZE\]\[DENSE_SIZE\]\s*=\s*\{(.*?)\};", re.DOTALL)
    dense_weights_match = dense_weights_pattern.search(content)
    if not dense_weights_match:
        raise ValueError("Dense weights not found in the file.")
    
    dense_weights_str = dense_weights_match.group(1)
    dense_weights = []
    for row_str in dense_weights_str.split("},"):
        row_weights = []
        for val_str in re.findall(r"-?\d+\.\d+", row_str):
            row_weights.append(float(val_str))
        if row_weights:
            dense_weights.append(row_weights)
    
    dense_weights = np.array(dense_weights)
    
    # Extract dense biases using regex
    dense_biases_pattern = re.compile(r"float dense_biases\s*\[DENSE_SIZE\]\s*=\s*\{(.*?)\};", re.DOTALL)
    dense_biases_match = dense_biases_pattern.search(content)
    if not dense_biases_match:
        raise ValueError("Dense biases not found in the file.")
    
    dense_biases_str = dense_biases_match.group(1)
    dense_biases = np.array([float(val_str) for val_str in re.findall(r"-?\d+\.\d+", dense_biases_str)])
    
    return dense_weights, dense_biases



def save_quantized_conv_to_file(file_path, quantized_conv_weights, quantized_conv_biases, bit_width, fractional_bits):
    """
    Save quantized convolutional weights and biases to the file.
    
    Args:
        file_path (str): Path to the file.
        quantized_conv_weights (np.ndarray): Quantized convolutional weights.
        quantized_conv_biases (np.ndarray): Quantized convolutional biases.
        bit_width (int): Total bit width of the fixed-point number.
        fractional_bits (int): Number of bits for the fractional part.
    """
    with open(file_path, 'w') as f:
        f.write("/*\n")
        f.write(" * This file is auto-generated by train.ipynb\n")
        f.write(" * Quantized to {}-bit fixed-point ({} fractional bits)\n".format(bit_width, fractional_bits))
        f.write(" */\n\n")
        f.write("#pragma once\n\n")
        f.write('#include "definitions.h"\n\n')
        
        # Write quantized convolutional weights
        f.write("// Conv layer weights.\n")
        f.write("int conv_weights [FILTERS][KRN_ROWS][KRN_COLS]\n")
        f.write("\t= {\n")
        for i in range(quantized_conv_weights.shape[0]):
            f.write("\t\t\t{\n")
            for j in range(quantized_conv_weights.shape[1]):
                f.write("\t\t\t\t{ ")
                for k in range(quantized_conv_weights.shape[2]):
                    f.write(str(quantized_conv_weights[i, j, k]))
                    if k < quantized_conv_weights.shape[2] - 1:
                        f.write(", ")
                f.write(" }")
                if j < quantized_conv_weights.shape[1] - 1:
                    f.write(",\n")
            f.write("\n\t\t\t}")
            if i < quantized_conv_weights.shape[0] - 1:
                f.write(",\n")
        f.write("\n\t\t};\n\n")
        
        # Write quantized convolutional biases
        f.write("// Conv layer biases.\n")
        f.write("int conv_biases [FILTERS] = { ")
        for i in range(quantized_conv_biases.shape[0]):
            f.write(str(quantized_conv_biases[i]))
            if i < quantized_conv_biases.shape[0] - 1:
                f.write(", ")
        f.write(" };\n")



def save_quantized_dense_to_file(file_path, quantized_dense_weights, quantized_dense_biases, bit_width, fractional_bits):
    """
    Save quantized dense weights and biases to the file.
    
    Args:
        file_path (str): Path to the file.
        quantized_dense_weights (np.ndarray): Quantized dense weights.
        quantized_dense_biases (np.ndarray): Quantized dense biases.
        bit_width (int): Total bit width of the fixed-point number.
        fractional_bits (int): Number of bits for the fractional part.
    """
    with open(file_path, 'w') as f:
        f.write("/*\n")
        f.write(" * This file is auto-generated by train.ipynb\n")
        f.write(" * Quantized to {}-bit fixed-point ({} fractional bits)\n".format(bit_width, fractional_bits))
        f.write(" */\n\n")
        f.write("#pragma once\n\n")
        f.write('#include "definitions.h"\n\n')
        
        # Write quantized dense weights
        f.write("// Dense layer weights.\n")
        f.write("int dense_weights[FLAT_SIZE][DENSE_SIZE]\n")
        f.write("\t= {\n")
        for i in range(quantized_dense_weights.shape[0]):
            f.write("\t\t\t{ ")
            for j in range(quantized_dense_weights.shape[1]):
                f.write(str(quantized_dense_weights[i, j]))
                if j < quantized_dense_weights.shape[1] - 1:
                    f.write(", ")
            f.write(" }")
            if i < quantized_dense_weights.shape[0] - 1:
                f.write(",\n")
        f.write("\n\t\t};\n\n")
        
        # Write quantized dense biases
        f.write("// Dense layer biases.\n")
        f.write("int dense_biases [DENSE_SIZE] = { ")
        for i in range(quantized_dense_biases.shape[0]):
            f.write(str(quantized_dense_biases[i]))
            if i < quantized_dense_biases.shape[0] - 1:
                f.write(", ")
        f.write(" };\n")

def quantize_weights_and_biases(weights, biases, bit_width, fractional_bits):
    """
    Quantize weights and biases to fixed-point representation.
    
    Args:
        weights (np.ndarray): The floating-point weights.
        biases (np.ndarray): The floating-point biases.
        bit_width (int): Total bit width of the fixed-point number.
        fractional_bits (int): Number of bits for the fractional part.
    
    Returns:
        tuple: Quantized weights and biases as numpy arrays.
    """
    quantized_weights = np.zeros_like(weights, dtype=int)
    quantized_biases = np.zeros_like(biases, dtype=int)
    
    # Quantize weights
    if len(weights.shape) == 3:  # Convolutional weights
        for i in range(weights.shape[0]):
            for j in range(weights.shape[1]):
                for k in range(weights.shape[2]):
                    quantized_weights[i, j, k] = float_to_fixed_point(weights[i, j, k], bit_width, fractional_bits)
    elif len(weights.shape) == 2:  # Dense weights
        for i in range(weights.shape[0]):
            for j in range(weights.shape[1]):
                quantized_weights[i, j] = float_to_fixed_point(weights[i, j], bit_width, fractional_bits)
    
    # Quantize biases
    for i in range(biases.shape[0]):
        quantized_biases[i] = float_to_fixed_point(biases[i], bit_width, fractional_bits)
    
    return quantized_weights, quantized_biases

In [44]:
# Example usage
# Define the bit width and fractional bits
bit_width = 32  # Total bit width
fractional_bits = 30  # Number of fractional bits

# Paths to the input files
conv_file_path = "Headers/conv_weights.h"
dense_file_path = "Headers/dense_weights.h"

conv_file_path_out = "Headers/Quantize/conv_weights.h"
dense_file_path_out = "Headers/Quantize/dense_weights.h"

# Parse and quantize convolutional weights and biases
# conv_weights, conv_biases = parse_conv_weights_and_biases(conv_file_path)
# quantized_conv_weights, quantized_conv_biases = quantize_weights_and_biases(conv_weights, conv_biases, bit_width, fractional_bits)
# save_quantized_conv_to_file(conv_file_path_out, quantized_conv_weights, quantized_conv_biases, bit_width, fractional_bits)

# Parse and quantize dense weights and biases
dense_weights, dense_biases = parse_dense_weights_and_biases(dense_file_path)
quantized_dense_weights, quantized_dense_biases = quantize_weights_and_biases(dense_weights, dense_biases, bit_width, fractional_bits)
save_quantized_dense_to_file(dense_file_path_out, quantized_dense_weights, quantized_dense_biases, bit_width, fractional_bits)

print(f"Quantization complete. Fixed-point weights and biases saved to {conv_file_path} and {dense_file_path}.")

Quantization complete. Fixed-point weights and biases saved to Headers/conv_weights.h and Headers/dense_weights.h.
